# HPC demo

# Introduction
This notebook shows how to use the distributed [raven/ostrich module](https://github.com/Ouranosinc/raven/wiki/Technical-Notes) on Compute Canada infrastructure. It runs a complete example of transferring local input data files required by raven/ostrich, running the tool remotely and retrieving the results.

# Design & strategy
Four issues arose during the module design stage:
* Authentication: how to access Compute Canada (CC) infrastructure
* Job submission: how to interact with CC's job scheduler
* Data transfer: how to copy the local data remotely and retrieve the generated output files
* Executable setup: how to configure a CC endpoint to invoke either _raven_/_ostrich_ binaries

## Authentication
The initial objective was to allow users to use their own credentials when running jobs on a CC cluster. This proved to be complicated because it would force users to upload their private keys to the server hosting the module (potential key compromission in the event of a security breach). A simpler approach has been adopted: create a _common_ user on CC side (called _crim01_) and execute jobs on its behalf.

## Job submission
Compute Canada does not provide rest APIs to allow job submissions from remote applications. Such tools exist but their deployment is rather a long-term plan on CC's roadmap. The unique solution is to take advantage of ssh's ability to execute commands on a remote machine, where commands in this context are the slurm utilities for job management (_sbatch_ for submission, _sacct_ for status enquiry and _scancel_ for job suppression). For our concern, jobs have four states: "PENDING" (sitting in the job queue), "RUNNING" (execution), "TIMEOUT" (job took too long to execute) and "CANCEL" (job was killed by the user).

## Data transfer
Again, the lack of public API imposes the use of "simple" tools such as _tar_ and _scp_ for data transfer. The series of steps include tarring the local input data (sitting in a directory whose path is provided to the module by the user), copying the tar file to crim01's working directory, untarring, running the job, tarring the results, copying the results tar file back to the caller (the module) and untarring it inside the directory provided by the user.

## Executable setup
The most elegant way to invoke either _raven_ or _ostrich_ on CC side is to launch a container that is configured to execute them. Predefined paths inside the container are mapped to subfolders in CRIM01's working directory and raven/ostrich are configured to make use of those predefined paths, for reading the parameter files and writing the output results. When thinking of containers, docker comes to mind but this technology is reportedly not appropriate for high-performance computing (https://dev.to/grokcode/singularity--a-docker-for-hpc-environments-i6p), so this is why Singularity has been chosen as container platform. Two Singularity images (one for _raven_, one for _ostrich_) are hosted in a public registry that has been set up at CRIM (132.217.141.54); when slurm schedules the job for execution (state RUNNING), the Singularity command-line tool pulls the appropriate image from the registry and executes it.

# Requirements
The module is easy to install and use, its single dependency being ParallelSSH (https://pypi.org/project/parallel-ssh/)

`$ pip install parallel_ssh`

In addition, the module expects the existence of a writable /tmp directory.

# Usage example
The follwing shows how to use the module: data is sent to a ComputeCanada cluster named cedar, a job involving either _raven_ or _ostrich_ is submitted and after execution (which may take a few minutes) the results are brought back locally. For this notebook, the default subdirectory 'test_data' contains examples for the mohyse-salmon dataset only.

_Note: be sure to execute cells only once, as job submission, an atomic operation by design, has been broken into many steps for demo purposes._

In [ ]:
# Directories
src_data_dir = "./"
out_dir = "./"
template_dir = "./template/"
executable = "raven"


def my_function(x):
    return x

In [ ]:
import datetime

# Executable selection
import os

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive

x = widgets.Dropdown(
    options=["raven", "ostrich"],
    value="raven",
    description="Executable",
    disabled=False,
)
wd = widgets.Dropdown(
    options=["mohyse-salmon"],  # ,'hmets-salmon', 'gr4j-salmon','hbv-salmon'],
    value="mohyse-salmon",
    description="Dataset",
    disabled=False,
)

w = interactive(my_function, x=x)

display(w)
d = interactive(my_function, x=wd)
display(d)

interactive(children=(Dropdown(description='Executable', options=('raven', 'ostrich'), value='raven'), Output(…

interactive(children=(Dropdown(description='Dataset', options=('mohyse-salmon',), value='mohyse-salmon'), Outp…

In [ ]:
executable = w.children[0].value
dataset = executable + "-" + d.children[0].value
print(f"{executable}: {dataset}")

raven: raven-mohyse-salmon


## Select data paths

Two paths must be provided to the module:
* The source path containing the input data files
* The output path that will hold the files generated by the executable on the cluster.

In [ ]:
wsrcpath = widgets.Text(
    value=os.path.join(os.getcwd(), "test_data"),
    placeholder="Source data path",
    description="Source data path:",
    disabled=False,
)
i_wsrcpath = interactive(my_function, x=wsrcpath)
display(i_wsrcpath)
woutpath = widgets.Text(
    value=os.path.join(os.getcwd(), "output"),
    placeholder="Output data path",
    description="Output data path:",
    disabled=False,
)
i_woutpath = interactive(my_function, x=woutpath)
display(i_woutpath)

interactive(children=(Text(value='/home/ets/src/raven2/raven/raven/hpc_interface/test_data', description='Sour…

interactive(children=(Text(value='/home/ets/src/raven2/raven/raven/hpc_interface/output', description='Output …

## Check connections
The module provides a function that performs two basic network connection checks:
* Is the access to the cluster allowed (i.e. can the module connect to its account)?
* Is the Singularity registry responding to a ping request?

Note: make sure that the path to the private key for ssh (supplied to the constructor of RavenHPCProcess) is good and that its permissions are [appropriate](http://www.sciencebits.com/Setting-SSH-Keys).

In [ ]:
src_data_dir = i_wsrcpath.children[0].value
out_dir = i_woutpath.children[0].value

import logging

import raven_process

logging.basicConfig(
    format="%(asctime)s %(message)s",
    level=logging.DEBUG,
    filename="hpclog.txt",
    filemode="w",
)
raven_proc = raven_process.RavenHPCProcess(
    executable,
    {"src_data_path": src_data_dir, "ssh_key_filename": "~/.ssh/pavics-hydro-crim01"},
)
status, msg = raven_proc.check_connection()
if status:
    print("Network connections are ok")
else:
    print("Network error: " + msg)

Network error: shub server down


## Estimate job duration

The job submission process requires the user to provide an *upper bound* estimate of the time needed to carry out the computations. 
* If the estimate is too low, the job may be killed before completion (when the computation time reaches the estimate)
* If the estimate is too high, the job may be scheduled with a lower priority.
Note that 20min will be added to the estimate to account for the download of the raven or ostrich singularity image.

In [ ]:
wduration = widgets.IntText(
    value=10, description="Job duration estimate (min):", disabled=False
)
i_wdur = interactive(my_function, x=wduration)
display(i_wdur)

interactive(children=(IntText(value=10, description='Job duration estimate (min):'), Output()), _dom_classes=(…

## Job summary before submission

In [ ]:
job_duration = str(datetime.timedelta(minutes=int(i_wdur.children[0].value) + 20))
print(f"Executable: {executable}")
print(f"Dataset: {dataset}")
print(f"Source data path: {src_data_dir}")
print(f"Output path: {out_dir}")
print(f"Time estimate: {job_duration}")

Executable: raven
Dataset: raven-mohyse-salmon
Source data path: /home/ets/src/raven2/raven/raven/hpc_interface/test_data
Output path: /home/ets/src/raven2/raven/raven/hpc_interface/output
Time estimate: 0:30:00


## Job submission
The job is ready to be submitted to slurm. Actual execution start time may be anywhere between a few seconds and many hours, depending on cluster load.

In [ ]:
# jobinfo = process_cmd(executable, client,hostname,"Submit")
print("Submitting job...")
try:
    raven_proc.submit(dataset, job_duration)
    print(f"Job ID: {raven_proc.live_job_id}")

except Exception as e:
    print(f"Job submission failed: {e}")

Submitting job...
Job ID: 23174203


## Polling for results
Once the job is queued, a monitoring function can be called to query its status (PENDING, RUNNING, etc.) When the job is in the RUNNING state, _raven_ or _ostrich_ is being executed and in both cases a progress figure (between 0% and 100%) is sent to the caller (here, the notebook cell). This figure may not always be available due to the fact that it is extracted from a text file being overwritten on a continuous basis.

The caller will typically invoke this monitoring function inside a while loop, leaving the loop upon termination of the execution (job completed or error); a reasonable call frequency could be one call per minute, as shown below.

In [ ]:
wprog = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description="Progress",
    bar_style="",  # 'success', 'info', 'warning', 'danger' or ''
    orientation="horizontal",
)
display(wprog)
wtext = widgets.HTML(
    value="",
    style={"description_width": "initial"},
    description="Received status",
    placeholder="Progress",
)
display(wtext)

i = 0
import time

job_finished = False
abnormal_ending = False
while not job_finished:
    time.sleep(60)
    try:
        out, p = raven_proc.monitor()
        # print(out)
        wtext.value += out + "<p>"
        if out == "RUNNING":
            # print("{}%".format(p))
            i += 1
            wprog.value = int(p)
        if out == "COMPLETED":
            job_finished = True
        if out == "TIMEOUT" or out == "CANCELLED":
            print("Uhoh: job " + out)
            abnormal_ending = True
            job_finished = True
        if out is None:
            print("Temp error")
        # Test job cancellation during job exec
        # if i==2:
        #    raven_proc.cancel()

    except Exception as e:
        print("Exception @monitor")
        print(e)
        # job_finished = True

# Check if job ended  normally
if abnormal_ending:
    print("Job ended abnormally")
    raven_proc.retrieve(out_dir)  # to get slurm log file

else:
    wprog.value = 100
    print("Retrieving results...")
    raven_proc.retrieve(out_dir)
    print(f"Results are ready: in {out_dir}")

raven_proc.cleanup()

IntProgress(value=0, description='Progress')

HTML(value='', description='Received status', placeholder='Progress', style=DescriptionStyle(description_width…

Retrieving results...
Results are ready: in /home/ets/src/raven2/raven/raven/hpc_interface/output


At that point, if everything went well (no errors in input data, no job cancellation, etc.), the execution results will be found in the output directory, along with the log file generated by the job scheduler (slurm).

# Additional notes
* Parallel invocations are supported: each module instance has its own workspace on ComputeCanada side, so many server sessions can manage jobs simultaneously.
* A nice advantage of the proposed implementation is the use of deployment: no requirements as far as ComputeCanada is concerned (no package to install or service to start), except an ssh connection and some user space.
* Limitation: module operation is bound by the rules in effect at ComputeCanada governing job management. For example, a directive issued in April 2019 announced a new rule that prevents user from submitting jobs from their /home directory; fixes to the module have been made but this kind of issue can still arise in the future.